In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
# import ray
# ray.init(num_cpus=4)
# import modin.pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter, defaultdict
import gc
import category_encoders as ce
import lightgbm as lgb
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
import datetime

from utils.schemas import *
from utils.functions import *

In [2]:
data_folder = 'input'

In [3]:
train = pd.read_csv(data_folder+'/train_generated_4.csv', dtype = schema_generated_4)#.sort_values('TransactionDT')
test = pd.read_csv(data_folder+'/test_generated_4.csv', dtype = schema_generated_4)#.sort_values('TransactionDT')

In [4]:
train.shape, test.shape

((590540, 538), (506691, 538))

In [5]:
START_DATE = datetime.datetime.strptime('2017-11-30', '%Y-%m-%d')

In [6]:
train['date'] = train['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))
test['date'] = test['TransactionDT'].apply(lambda x: (START_DATE + datetime.timedelta(seconds = x)))

train['month'] = train['date'].dt.month
test['month'] = test['date'].dt.month

train['day'] = train['date'].dt.day
test['day'] = test['date'].dt.day

train['dayofweek'] = train['date'].dt.dayofweek
test['dayofweek'] = test['date'].dt.dayofweek

train['hour'] = train['date'].dt.hour
test['hour'] = test['date'].dt.hour

In [7]:
train_index = train.shape[0]
test_index = test.shape[0]
df = pd.concat([train, test], axis=0)
df.shape
# del train, test
gc.collect()

7

In [8]:
old_cols = df.columns

In [9]:
# groups V
cols1 = ['V{}'.format(x) for x in range(1,12)]
cols2 = ['V{}'.format(x) for x in range(12,35)]
cols3 = ['V{}'.format(x) for x in range(35,53)]
cols4 = ['V{}'.format(x) for x in range(53,75)]
cols5 = ['V{}'.format(x) for x in range(75,95)]
cols6 = ['V{}'.format(x) for x in range(95,138)]
cols7 = ['V{}'.format(x) for x in range(138,167)]
cols8 = ['V{}'.format(x) for x in range(167,217)]
cols9 = ['V{}'.format(x) for x in range(217,279)]
cols10 = ['V{}'.format(x) for x in range(279,322)]
cols11 = ['V{}'.format(x) for x in range(322,340)]

In [10]:
# Binary V
bin_v = ['V1', 'V14', 'V41', 'V65', 'V88', 'V107', 'V305']

In [11]:
V_cols = [cols1, cols2, cols3, cols4, cols5, cols6, cols7, cols8, cols9, cols10, cols11, bin_v]

In [12]:
def freq_encoder2(df, label, min_freq = 0.001):
    rows = df.shape[0]
    n = 0
    dict_fe = dict()
    vc = df[label].value_counts()
    for i, j in zip(vc.index, vc):
        ratio = j/rows
        if ratio > min_freq:
            dict_fe[i] = n
            n += 1
        else:
            dict_fe[i] = n

    if n < 2**8:
        _d_type = 'uint8'
    elif n >= 2**8 and n < 8**16:
        _d_type = 'uint16'
    elif n >= 2**16 and n < 8**32:
        _d_type = 'uint32'
    else:
        _d_type = 'uint64'

#     df[new_label] = df[label].apply(lambda x: dict_fe[x]).astype(_d_type)

    return dict_fe, _d_type

def freq_encoder2_test(df, label, dict_fe):
    list_unique = df[label].unique()
    max_val = max(dict_fe.values())
    train_values = dict_fe.keys()
    for v in list_unique:
        if v not in train_values:
            dict_fe[v] = max_val+1
    
    return dict_fe

In [13]:
def reduce_memory2(df):
    print("Reduce_memory...");
    dict_types = dict()
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    dict_types[col] = 'int8'
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    dict_types[col] = 'int16'
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    dict_types[col] = 'int32'
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    dict_types[col] = 'int64'
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    dict_types[col] = 'float16'
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    dict_types[col] = 'float32'
                else:
                    dict_types[col] = 'float64'
    return dict_types


In [14]:
def plot_col(df, col, scat=True):
    if scat:
        fig, ax = plt.subplots()
        fig.set_size_inches(15, 6)
        ax.scatter(df[df.isFraud == 0].TransactionDT, df[df.isFraud == 0][col])
        ax.scatter(df[df.isFraud == 1].TransactionDT, df[df.isFraud == 1][col])
        ax.scatter(df[df.isFraud.isna()].TransactionDT, df[df.isFraud.isna()][col])
    else:
        fig, ax = plt.subplots()
        fig.set_size_inches(15, 6)
        ax.plot(df[df.isFraud == 0].TransactionDT, df[df.isFraud == 0][col])
        ax.plot(df[df.isFraud == 1].TransactionDT, df[df.isFraud == 1][col])
        ax.plot(df[df.isFraud.isna()].TransactionDT, df[df.isFraud.isna()][col])
    plt.show()
    gc.collect()

In [15]:
def plot_hist_col(df, col):
    fig, ax = plt.subplots()
    fig.set_size_inches(15, 6)
    ax.hist(df[df.isFraud.isna()][col], 100)
    ax.hist(df[df.isFraud == 0][col], 100)
    ax.hist(df[df.isFraud == 1][col], 100)
    plt.show()
    gc.collect()

In [16]:
def micro_train(X_fit, y_fit, X_val, y_val):
    params = {
#     'num_leaves': 256,
    'max_depth': 9,
#     'max_leaf_nodes': 45,
#     'min_sample_leaf': 20,
    'metric': ['AUC'],
    'first_metric_only': True,
    'n_estimators': 50000,
    'num_threads': 8,
    'learning_rate': 0.01,
    'colsample_bytree': 0.5,
    'objective': 'xentropy',
    'bagging_fraction': 0.5,
    'bagging_freq': 5,
    'importance_type': 'gain',
    'bagging_seed': 42,
    'random_state':42,
    'seed': 42,
    'feature_fraction_seed': 42,
    'drop_seed': 42,
    'data_random_seed': 42,
    }
    model = lgb.LGBMClassifier(**params)
    model.fit(X_fit, y_fit, eval_set=[(X_val, y_val), (X_fit, y_fit)],
         early_stopping_rounds=50, verbose=50)
    
    df_importances = pd.DataFrame({'feature': X_fit.columns, 'importance': model.feature_importances_})\
    .sort_values('importance', ascending=False).reset_index(drop=True)
    
    return df_importances

In [17]:
first_drop_col = ['D9','id_24','id_25','id_07','id_08','id_21','id_26','id_27','id_23','id_22','dist2']

In [18]:
cols_v = [x for x in train.columns if 'V' in x]

In [19]:
cols_c = [x for x in df.columns if 'C' in x and len(x)<4]

In [20]:
cols_d = [x for x in df.columns if 'D' in x and len(x)<4]

In [21]:
cols_m = [x for x in df.columns if 'M' in x and len(x) < 4]

In [22]:
# new_cat_features = list()
# for c in train.columns:
#     if train[c].nunique() < 50 and c not in cat_ft:
#         new_cat_features.append(c)
        
# exclude_list = ['isFraud','month','date_fe1''day','dayofweek',
#                 'has_identity','hour','date_fe1','PCA_28','PCA_29']

# new_cat_features = [x for x in new_cat_features if x not in exclude_list]

In [116]:
counter = 0
cols_c_c = list()
for i,j in enumerate(cols_c):
    for k in cols_c[i:]:
        if k != j:
#             print('Variables: {0} entre {1}'.format(j, k))
            df['r_{}'.format(counter)] = np.log1p(df[j]/df[k]).replace(np.inf, np.nan)#.clip(-100,100)
            cols_c_c.append('r_{}'.format(counter))
            counter += 1
# train['r'] = (train.C1/train.C13).replace(np.inf, np.nan).clip(-100,100)

In [23]:
print('Cols D')
for c in cols_d:
    df['{}_group_1'.format(c)] = df[c] / df.groupby(['card1','month'])[c].transform('mean')
    df['{}_group_2'.format(c)] = df[c] / df.groupby(['card1','month'])[c].transform('count')
    df['{}_group_3'.format(c)] = df[c] / df.groupby(['card1','month'])[c].transform('sum')
    
print('Cols C')
for c in cols_c:
    df['{}_group_1'.format(c)] = df[c] / df.groupby(['card1','month'])[c].transform('mean')
    df['{}_group_2'.format(c)] = df[c] / df.groupby(['card1','month'])[c].transform('count')
    df['{}_group_3'.format(c)] = df[c] / df.groupby(['card1','month'])[c].transform('sum')

Cols D
Cols C


In [24]:
print('Cols D')
for c in cols_d:
    df['{}_group_4'.format(c)] = df[c] / df.groupby(['card1','card2','card3',
                                                     'card4','card5','card6','month'])[c].transform('mean')
    df['{}_group_5'.format(c)] = df[c] / df.groupby(['card1','card2','card3',
                                                     'card4','card5','card6','month'])[c].transform('count')
    df['{}_group_6'.format(c)] = df[c] / df.groupby(['card1','card2','card3',
                                                     'card4','card5','card6','month'])[c].transform('sum')
    
print('Cols C')
for c in cols_c:
    df['{}_group_4'.format(c)] = df[c] / df.groupby(['card1','card2','card3',
                                                     'card4','card5','card6','month'])[c].transform('mean')
    df['{}_group_5'.format(c)] = df[c] / df.groupby(['card1','card2','card3',
                                                     'card4','card5','card6','month'])[c].transform('count')
    df['{}_group_6'.format(c)] = df[c] / df.groupby(['card1','card2','card3',
                                                     'card4','card5','card6','month'])[c].transform('sum')

Cols D
Cols C


In [25]:
gc.collect()

0

In [26]:
grouped_columns = [['card1', 'month'], ['card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'month']]

In [28]:
df.M1.value_counts()

0    649436
1    447739
2        56
Name: M1, dtype: int64

In [33]:
df['M1_1'] = np.where(df.M1 == 1, 1, 0)

In [35]:
df['M1_1_sum'] = df.groupby(grouped_columns[0])['M1_1'].transform('sum')

In [36]:
df['N0'] = df.groupby(grouped_columns[0])['TransactionAmt'].transform('nunique')
df['N1'] = df.groupby(grouped_columns[0])['card2'].transform('nunique')
df['N2'] = df.groupby(grouped_columns[0]+['card2'])['card3'].transform('nunique')
df['N3'] = df.groupby(grouped_columns[0]+['day'])['TransactionAmt'].transform('nunique')
df['N4'] = df.TransactionAmt / df.groupby(grouped_columns[0])['TransactionAmt'].transform('mean')
df['N5'] = df.groupby(grouped_columns[0])['addr1'].transform('nunique')
df['N6'] = df.groupby(grouped_columns[0] + ['addr1'])['addr2'].transform('nunique')
df['TransactionDT_diff0'] = (df.TransactionDT - df.groupby(grouped_columns[0])['TransactionDT'].shift(1))\
.replace(np.inf, np.nan)

df['N7'] = df.groupby(grouped_columns[1])['TransactionAmt'].transform('nunique')
df['N8'] = df.groupby(grouped_columns[1]+['day'])['TransactionAmt'].transform('nunique')
df['N9'] = df.groupby(grouped_columns[0])['addr1'].transform('nunique')
df['N10'] = df.groupby(grouped_columns[0] + ['addr1'])['addr2'].transform('nunique')
df['N11'] = df.TransactionAmt / df.groupby(grouped_columns[1])['TransactionAmt'].transform('mean')
df['TransactionDT_diff1'] = (df.TransactionDT - df.groupby(grouped_columns[1])['TransactionDT'].shift(1))\
.replace(np.inf, np.nan)
gc.collect()

0

In [39]:
cut_train_1 = train[(train.month < 4) | (train.month == 12)].shape[0]
cut_train_2 = train[(train.month > 4) & (train.month != 12)].index[0]
cut_train_1, cut_train_2

(417559, 501214)

In [59]:
drop_cols_new = ['TransactionDT', 'date', 'day', 'month','isFraud', 'date_fe1',
                 'addr1',
                 'addr2',
                 'card1',
                 'card2',
                 'card3',
                 'card4',
                 'card5',
                 'card6',
                 'card1_fe1',
                 'card2_fe1',
                 'card3_fe1',
                 'card4_fe1',
                 'card5_fe1',
                 'card6_fe1',
                 'addr1_fe1',
                 'addr2_fe1',
                 'M1_fe1',
                 'M2_fe1',
                 'M3_fe1',
                 'M4_fe1',
                 'M5_fe1',
                 'M6_fe1',
                 'M7_fe1',
                 'M8_fe1',
                 'M9_fe1'
                ] + first_drop_col + cols_c + cols_d + cols_v + cols_m

In [60]:
select_cols = [x for x in df.columns if x not in drop_cols_new+['addr']]

In [61]:
# X = pd.concat([pd.DataFrame(df_pca[:len(train)]), train[cols_pca]],axis=1)
# X = pd.concat([df[cols_c_c][:len(train)], train[cols_pca]],axis=1)
X = df[select_cols][:len(train)]
y = train.isFraud
gc.collect()

183

In [56]:
X_fit = X[:cut_train_1]
y_fit = train.isFraud[:cut_train_1]

X_val = X[cut_train_2:]
y_val = train.isFraud[cut_train_2:]
gc.collect()

df_importances = micro_train(X_fit, y_fit, X_val, y_val)
# [2944]	training's auc: 0.975654	valid_0's auc: 0.920092

Training until validation scores don't improve for 50 rounds.
[50]	training's auc: 0.873373	valid_0's auc: 0.859318
[100]	training's auc: 0.883265	valid_0's auc: 0.865073
[150]	training's auc: 0.889467	valid_0's auc: 0.868435
[200]	training's auc: 0.896512	valid_0's auc: 0.871728
[250]	training's auc: 0.903188	valid_0's auc: 0.874631
[300]	training's auc: 0.910268	valid_0's auc: 0.877107
[350]	training's auc: 0.916063	valid_0's auc: 0.879472
[400]	training's auc: 0.921724	valid_0's auc: 0.881658
[450]	training's auc: 0.925274	valid_0's auc: 0.883361
[500]	training's auc: 0.928929	valid_0's auc: 0.884541
[550]	training's auc: 0.932423	valid_0's auc: 0.885961
[600]	training's auc: 0.935033	valid_0's auc: 0.887023
[650]	training's auc: 0.937868	valid_0's auc: 0.888473
[700]	training's auc: 0.940495	valid_0's auc: 0.889327
[750]	training's auc: 0.942827	valid_0's auc: 0.889866
[800]	training's auc: 0.945223	valid_0's auc: 0.890449
[850]	training's auc: 0.947369	valid_0's auc: 0.890854
[900

In [57]:
df_importances.head(50)

,feature,importance
0,PCA_27,357745.250583
1,C1,94157.222242
2,PCA_23,85776.800446
3,C14,44595.274899
4,D2,35948.781884
5,PCA_18,35605.378831
6,TransactionAmt,35568.416715
7,PCA_16,34264.652002
8,C13,29231.824489
9,PCA_26,26856.580355


In [64]:
X_fit, X_val, y_fit, y_val = train_test_split(X, y, test_size=0.33, random_state=42)
df_importances = micro_train(X_fit, y_fit, X_val, y_val)
gc.collect()

Training until validation scores don't improve for 50 rounds.
[50]	training's auc: 0.867006	valid_0's auc: 0.866718
[100]	training's auc: 0.877174	valid_0's auc: 0.87573
[150]	training's auc: 0.884448	valid_0's auc: 0.881819
[200]	training's auc: 0.891095	valid_0's auc: 0.88712
[250]	training's auc: 0.897202	valid_0's auc: 0.892008
[300]	training's auc: 0.903991	valid_0's auc: 0.896918
[350]	training's auc: 0.909048	valid_0's auc: 0.900849
[400]	training's auc: 0.914175	valid_0's auc: 0.905038
[450]	training's auc: 0.918713	valid_0's auc: 0.908277
[500]	training's auc: 0.92276	valid_0's auc: 0.910824
[550]	training's auc: 0.926188	valid_0's auc: 0.91307
[600]	training's auc: 0.929339	valid_0's auc: 0.915423
[650]	training's auc: 0.932136	valid_0's auc: 0.917351
[700]	training's auc: 0.934996	valid_0's auc: 0.919108
[750]	training's auc: 0.937122	valid_0's auc: 0.920403
[800]	training's auc: 0.939182	valid_0's auc: 0.921785
[850]	training's auc: 0.94135	valid_0's auc: 0.922918
[900]	tra

[7400]	training's auc: 0.997242	valid_0's auc: 0.957568
[7450]	training's auc: 0.997303	valid_0's auc: 0.957621
[7500]	training's auc: 0.997331	valid_0's auc: 0.957661
[7550]	training's auc: 0.997393	valid_0's auc: 0.957729
[7600]	training's auc: 0.997448	valid_0's auc: 0.957786
[7650]	training's auc: 0.997494	valid_0's auc: 0.957839
[7700]	training's auc: 0.99753	valid_0's auc: 0.95785
[7750]	training's auc: 0.997588	valid_0's auc: 0.957928
[7800]	training's auc: 0.997631	valid_0's auc: 0.957967
[7850]	training's auc: 0.997689	valid_0's auc: 0.958058
[7900]	training's auc: 0.997742	valid_0's auc: 0.958108
[7950]	training's auc: 0.99781	valid_0's auc: 0.958182
[8000]	training's auc: 0.997867	valid_0's auc: 0.958262
[8050]	training's auc: 0.997923	valid_0's auc: 0.958293
[8100]	training's auc: 0.997972	valid_0's auc: 0.958292
[8150]	training's auc: 0.998035	valid_0's auc: 0.95836
[8200]	training's auc: 0.998071	valid_0's auc: 0.95843
[8250]	training's auc: 0.99812	valid_0's auc: 0.95848

162

In [65]:
df_importances.head(50)

,feature,importance
0,PCA_27,466695.785258
1,PCA_23,74470.727741
2,TransactionAmt,68216.642766
3,PCA_26,67687.371984
4,PCA_18,46983.138782
5,PCA_16,40806.333092
6,PCA_24,36545.861892
7,N4,32865.598697
8,C1_group_4,32156.845833
9,C14_group_1,30889.460238


In [93]:
df.dtypes.to_frame('dtyp')

,dtyp
PCA_0,float16
PCA_1,float16
PCA_2,float16
PCA_3,float16
PCA_4,float16
...,...
N8,float16
N9,uint16
N10,uint8
N11,float16


In [66]:
new_cols = list(set(df.columns) - set(old_cols))
len(new_cols)

190

In [67]:
dict_types = reduce_memory2(df[new_cols])

Reduce_memory...


In [40]:
df[new_cols] = df[new_cols].astype(dict_types)

In [ ]:
# df = df[[x for x in df.columns if x not in drop_cols]]

In [ ]:
# df = df.replace(np.inf, np.nan)

In [94]:
train = df.iloc[:train_index, :]
test = df.iloc[train_index:, :]
train.shape, test.shape

((590540, 728), (506691, 728))

In [95]:
train.to_csv(data_folder+'/train_ft_eng_4.csv', header=True, index=None)
print('train guardado')
test.to_csv(data_folder+'/test_ft_eng_4.csv', header=True, index=None)
print('test guardado')

train guardado
test guardado


In [51]:
df.columns

Index(['PCA_0', 'PCA_1', 'PCA_2', 'PCA_3', 'PCA_4', 'PCA_5', 'PCA_6', 'PCA_7',
       'PCA_8', 'PCA_9',
       ...
       'date_fe1', 'R_emaildomain_1_fe1', 'proc_id_30_0_fe1',
       'proc_id_30_1_fe1', 'proc_id_31_0_fe1', 'P_emaildomain_0_fe1',
       'device_name_fe1', 'R_emaildomain_0_fe1', 'P_emaildomain_1_fe1',
       'device_version_fe1'],
      dtype='object', length=538)

Mirar para este "usuario" q variables son iguales los target = 1 de los target = 2 consecutivos para determinar que es un usuario de verdad

In [82]:
df[(df.card1 == 19)&(df.card2==13)&(df.card3==1)&(df.ProductCD==1)][['date','card1','card2','card3','card4','card5','card5','TransactionAmt', 'ProductCD','isFraud']].head(20)

,date,card1,card2,card3,card4,card5,card5,TransactionAmt,ProductCD,isFraud
412,2017-12-01 01:51:42,19,13,1,1,1,1,35.031250,1,0.0
499,2017-12-01 02:21:01,19,13,1,1,1,1,9.421875,1,0.0
577,2017-12-01 02:48:24,19,13,1,1,1,1,6.476562,1,0.0
584,2017-12-01 02:51:07,19,13,1,1,1,1,32.312500,1,0.0
630,2017-12-01 03:10:43,19,13,1,1,1,1,12.328125,1,1.0
683,2017-12-01 03:39:44,19,13,1,1,1,1,124.375000,1,1.0
750,2017-12-01 04:03:56,19,13,1,1,1,1,60.062500,1,0.0
886,2017-12-01 05:52:18,19,13,1,1,1,1,36.375000,1,0.0
1549,2017-12-01 13:37:50,19,13,1,1,1,1,25.250000,1,0.0
2887,2017-12-01 17:48:06,19,13,1,1,1,1,90.250000,1,0.0


In [78]:
df[(df.card1 == 321)&(df.card2==30)&(df.card3==1)&(df.ProductCD==1)&(df.isFraud==1)][['date','card1','card2','card3','card4','card5','card5','TransactionAmt', 'ProductCD','isFraud']]

,date,card1,card2,card3,card4,card5,card5,TransactionAmt,ProductCD,isFraud
240,2017-12-01 01:03:13,321,30,1,0,8,8,37.09375,1,1.0
243,2017-12-01 01:04:06,321,30,1,0,8,8,37.09375,1,1.0
245,2017-12-01 01:04:55,321,30,1,0,8,8,37.09375,1,1.0
21902,2017-12-06 16:25:31,321,30,1,0,8,8,60.71875,1,1.0
25956,2017-12-07 17:55:44,321,30,1,0,8,8,79.00000,1,1.0
...,...,...,...,...,...,...,...,...,...,...
561637,2018-05-21 01:06:57,321,30,1,0,8,8,118.31250,1,1.0
561743,2018-05-21 01:49:26,321,30,1,0,8,8,59.15625,1,1.0
562030,2018-05-21 04:46:28,321,30,1,0,8,8,118.31250,1,1.0
566529,2018-05-22 20:04:57,321,30,1,0,8,8,119.18750,1,1.0


In [80]:
df[(df.card1 == 76)&(df.card2==4)&(df.card3==0)&(df.isFraud==1)][['date','card1','card2','card3','card4','card5','card5','TransactionAmt', 'ProductCD','isFraud']]

,date,card1,card2,card3,card4,card5,card5,TransactionAmt,ProductCD,isFraud
203,2017-12-01 00:56:00,76,4,0,0,0,0,445.0000,0,1.0
11766,2017-12-03 22:47:26,76,4,0,0,0,0,400.0000,0,1.0
12324,2017-12-04 00:58:31,76,4,0,0,0,0,225.0000,2,1.0
23099,2017-12-06 21:15:36,76,4,0,0,0,0,400.0000,0,1.0
24198,2017-12-07 01:38:44,76,4,0,0,0,0,171.0000,0,1.0
24218,2017-12-07 01:42:48,76,4,0,0,0,0,171.0000,0,1.0
25256,2017-12-07 15:07:37,76,4,0,0,0,0,250.0000,0,1.0
25277,2017-12-07 15:14:24,76,4,0,0,0,0,272.0000,0,1.0
28128,2017-12-08 02:37:11,76,4,0,0,0,0,330.0000,0,1.0
30979,2017-12-08 21:50:07,76,4,0,0,0,0,107.9375,0,1.0


In [35]:
# # def process_data(df):
# print('Processing joined data...')
# df['M'] = df.M1.astype(str) + df.M2.astype(str) + df.M3.astype(str) +\
# df.M4.astype(str) + df.M5.astype(str) + df.M6.astype(str) + \
# df.M7.astype(str) + df.M8.astype(str) + df.M9.astype(str)

# df = df.drop(['D9','id_24','id_25','id_07','id_08','id_21','id_26','id_27','id_23','id_22','dist2'],axis=1)

# df['email'] = df.P_emaildomain.astype(str) + df.R_emaildomain.astype(str)

# #     df['Transaction_day_of_week'] = np.floor((df['TransactionDT'] / (3600 * 24) - 1) % 7)
# #     df['Transaction_hour'] = np.floor(df['TransactionDT'] / 3600) % 24

# binary_cat_ft = ['card', 'M', 'email', 'dayofweek', 'hour', 'DeviceType',
#                  'proc_id_30_0', 'proc_id_30_1', 'proc_id_31_0']
# encoder = ce.BinaryEncoder(cols=binary_cat_ft)
# df = encoder.fit_transform(df)


# #     df['Transaction_day_of_week'] = np.floor((df['TransactionDT'] / (3600 * 24) - 1) % 7)
# #     df['day'] = np.floor((df['TransactionDT'] / (3600 * 24) - 1))
# #     df['Transaction_hour'] = np.floor(df['TransactionDT'] / 3600) % 24
# #     dict_fe, _d_type = freq_encoder2(df, 'Transaction_day_of_week', 0)
# #     df['Transaction_day_of_week'] = df['Transaction_day_of_week'].apply(lambda x: dict_fe[x]).astype(_d_type)
# #     dict_fe, _d_type = freq_encoder2(df, 'Transaction_hour', 0)
# #     df['Transaction_hour'] = df['Transaction_hour'].apply(lambda x: dict_fe[x]).astype(_d_type)

# id_cols = ['card1', 'card2', 'month']
# cols1 = [x for x in df.columns if 'card' in x and '_fe1' in x]

# cols1 = [x for x in df.columns if 'card' in x and '_fe1' in x]+['month']
# ft = 'N1'
# df[ft] = df.groupby(cols1)['TransactionDT'].transform('count')
# dict_fe, _d_type = freq_encoder2(df, ft, min_freq = 0)
# df[ft] = df[ft].apply(lambda x: dict_fe[x]).astype(_d_type)

# cols1 = ['addr1', 'addr2']+['month']
# ft = 'N2'
# df[ft] = df.groupby(cols1)['TransactionDT'].transform('count')
# dict_fe, _d_type = freq_encoder2(df, ft, min_freq = 0)
# df[ft] = df[ft].apply(lambda x: dict_fe[x]).astype(_d_type)

# cols1 = [x for x in df.columns if 'C' in x and 'Product' not in x]+['month']
# ft = 'N3'
# df[ft] = (df.groupby(cols1)['TransactionDT'].transform('count')).fillna(-1)
# dict_fe, _d_type = freq_encoder2(df, ft, min_freq = 0)
# df[ft] = df[ft].apply(lambda x: dict_fe[x]).astype(_d_type)

# cols1 = [x for x in df.columns if 'mail' in x]+['month']
# ft = 'N4'
# df[ft] = (df.groupby(cols1)['TransactionDT'].transform('count')).fillna(-1)
# dict_fe, _d_type = freq_encoder2(df, ft, min_freq = 0)
# df[ft] = df[ft].apply(lambda x: dict_fe[x]).astype(_d_type)

# cols1 = ['P_emaildomain', 'R_emaildomain', 'addr1']+['month']
# ft = 'N5'
# df[ft] = df.groupby(cols1)['TransactionDT'].transform('count')
# dict_fe, _d_type = freq_encoder2(df, ft, min_freq = 0)
# df[ft] = df[ft].apply(lambda x: dict_fe[x]).astype(_d_type)


# cols1 = ['addr1', 'addr2', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6']+['month']
# ft = 'N6'
# df[ft] = df.groupby(cols1)['TransactionDT'].transform('count')
# dict_fe, _d_type = freq_encoder2(df, ft, min_freq = 0)
# df[ft] = df[ft].apply(lambda x: dict_fe[x]).astype(_d_type)

# c_cols = [x for x in df.columns if 'C' in x and x not in ['ProductCD','ProductCD_fe1','ProductCD_fe2']]

# df['R1'] = (np.log1p(df.C13/df.C1)).replace(np.inf, np.nan)
# df['R1_2'] = (np.log1p(df.C2/df.C1)).replace(np.inf, np.nan)
# df['R1_3'] = (np.log1p(df.C3/df.C1)).replace(np.inf, np.nan)
# df['R1_4'] = (np.log1p(df.C4/df.C1)).replace(np.inf, np.nan)
# df['R1_5'] = (np.log1p(df.C5/df.C1)).replace(np.inf, np.nan)

# df['R2'] = (np.log1p(df.D1/df.D15)).replace(np.inf, np.nan)

# df['R3'] = (np.log1p(df.TransactionAmt/df.TransactionAmt.values.mean())).replace(np.inf, np.nan)

# df['R4'] = (np.log1p(df.N3/df.N1)).replace(np.inf, np.nan)

# df['R5'] = (df['D15'] / df.groupby(['card1', 'addr', 'month'])['D15'].transform('std')).replace(np.inf, np.nan)

# df['R6'] = (df['D15'] / df.groupby(['TransactionDT', 'month'])['D15'].transform('std')).replace(np.inf, np.nan)

# df['R7'] = (df.TransactionAmt / df.hour / df.dayofweek).replace(np.inf, np.nan)

# df['R8'] = (df['TransactionAmt'] / df.groupby(['card1', 'addr1', 'month'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

# df['R9'] = (df['TransactionAmt'] / df.groupby(['card1','C13','C1','card2', 'month'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

# df['R10'] = (df['TransactionAmt'] / df.groupby(['card1','addr1','id_01','V258','D1', 'P_emaildomain', 'R_emaildomain', 'month'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

# df['R11'] = (df['TransactionAmt'] / df.groupby(['card1','addr1','P_emaildomain', 'R_emaildomain', 'month'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

# df['R12'] = (df.groupby(['card1','addr1', 'month'])['card1'].transform('mean') * df.groupby(['card1','addr1', 'month'])['card1'].transform('count')).replace(np.inf, np.nan)

# df['R13'] = (df.TransactionAmt / df.groupby(['card1', 'card2','addr1', 'month'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

# df['R14'] = (df.C2 / df.groupby(['card1', 'V258', 'D10', 'month'])['C2'].transform('mean')).replace(np.inf, np.nan)

# df['R15'] = (df.V130 / df.groupby(['addr1', 'card1', 'C10', 'month'])['V130'].transform('std')).replace(np.inf, np.nan)

# df['R16'] = (df.V258 / df.groupby(['addr1', 'card1', 'C10', 'month'])['V258'].transform('std')).replace(np.inf, np.nan)

# df['R17'] = (df.D10 / df.groupby(id_cols)['D10'].transform('std')).replace(np.inf, np.nan)

# df['R18'] = (df.D11 / df.groupby(id_cols)['D11'].transform('std')).replace(np.inf, np.nan)

# df['R19'] = (df.D15 / df.groupby(id_cols)['D15'].transform('std')).replace(np.inf, np.nan)

# df['R20'] = (df.D3 / df.groupby(id_cols)['D3'].transform('std')).replace(np.inf, np.nan)

# df['R21'] = (df.D4 / df.groupby(id_cols)['D4'].transform('std')).replace(np.inf, np.nan)

# df['R22'] = (df.D5 / df.groupby(id_cols)['D5'].transform('std')).replace(np.inf, np.nan)

# df['R23'] = (df.D6 / df.groupby(id_cols)['D6'].transform('std')).replace(np.inf, np.nan)

# df['R24'] = df.D7 * df.groupby(id_cols)['D7'].transform('std')
# df['R24'] = df['R24'].replace(np.inf, np.nan)
# df['R24'] = df['R24']/df['R24'].max()

# df['R25'] = df.D12 * df.groupby(id_cols)['D12'].transform('std')
# df['R25'] = df['R25'].replace(np.inf, np.nan)
# df['R25'] = df['R25']/df['R25'].max()

# df['R26'] = df.D6 * df.groupby(id_cols)['D6'].transform('std')
# df['R26'] = df['R26'].replace(np.inf, np.nan)
# df['R26'] = df['R26']/df['R26'].max()

# df['R27'] = df.D8 * df.groupby(id_cols)['D8'].transform('std')
# df['R27'] = df['R27'].replace(np.inf, np.nan)
# df['R27'] = df['R27']/df['R27'].max()

# df['R28'] = (df.TransactionAmt - df.groupby(id_cols)['TransactionAmt'].shift(1)).replace(np.inf, np.nan)

# df['R29'] = (df.TransactionDT - (df.TransactionDT - df.groupby(id_cols)['TransactionDT'].shift(1))).replace(np.inf, np.nan)

# df['R30'] = (df.dist1 - df.groupby(id_cols)['dist1'].shift(1)).replace(np.inf, np.nan)

# df['R31'] = (df.dist2 - df.groupby(id_cols)['dist2'].shift(1)).replace(np.inf, np.nan)

n = 32
for c in c_cols:
    df['R{}'.format(n)] = (df[c] / df.groupby(id_cols)[c].transform('std')).replace(np.inf, np.nan)
    n += 1

for c in c_cols:
    df['R{}'.format(n)] = (df[c] / df.groupby(id_cols)[c].transform('mean')).replace(np.inf, np.nan)
    n += 1

for c in c_cols:
    df['R{}'.format(n)] = (df[c] / (df.groupby(id_cols)[c].transform('std')/df.groupby(id_cols)[c].transform('mean'))).replace(np.inf, np.nan)
    n += 1


gc.collect()

print('End!\n')

#     return df

End!



In [11]:
def process_data2(train, test):
    print('Processing train and test data...')
    train['M'] = train.M1.astype(str) + train.M2.astype(str) + train.M3.astype(str) +\
    train.M4.astype(str) + train.M5.astype(str) + train.M6.astype(str) + \
    train.M7.astype(str) + train.M8.astype(str) + train.M9.astype(str)

    test['M'] = test.M1.astype(str) + test.M2.astype(str) + test.M3.astype(str) +\
    test.M4.astype(str) + test.M5.astype(str) + test.M6.astype(str) + \
    test.M7.astype(str) + test.M8.astype(str) + test.M9.astype(str)

    train = train.drop(['D9'],axis=1)

    test = test.drop(['D9'],axis=1)

    train['card'] = train.card1.astype(str) + train.card2.astype(str) + train.card3.astype(str) +\
    train.card4.astype(str) + train.card5.astype(str) + train.card6.astype(str)

    test['card'] = test.card1.astype(str) + test.card2.astype(str) + test.card3.astype(str) +\
    test.card4.astype(str) + test.card5.astype(str) + test.card6.astype(str)

    train['email'] = train.P_emaildomain.astype(str) + train.R_emaildomain.astype(str)

    test['email'] = test.P_emaildomain.astype(str) + test.R_emaildomain.astype(str)

    train['Transaction_day_of_week'] = np.floor((train['TransactionDT'] / (3600 * 24) - 1) % 7)
    train['Transaction_hour'] = np.floor(train['TransactionDT'] / 3600) % 24
    train['day'] = np.floor((train['TransactionDT'] / (3600 * 24) - 1))
    
    test['day'] = np.floor((test['TransactionDT'] / (3600 * 24) - 1))
    test['Transaction_day_of_week'] = np.floor((test['TransactionDT'] / (3600 * 24) - 1) % 7)
    test['Transaction_hour'] = np.floor(test['TransactionDT'] / 3600) % 24

    binary_cat_ft = ['card', 'M', 'email', 'Transaction_day_of_week', 'Transaction_hour', 'DeviceType', 'proc_id_30_0',
                     'proc_id_30_1', 'proc_id_31_0']

    encoder = ce.BinaryEncoder(cols=binary_cat_ft)
    encoder.fit(train)
    train = encoder.transform(train)
    test = encoder.transform(test)

    train['Transaction_day_of_week'] = np.floor((train['TransactionDT'] / (3600 * 24) - 1) % 7)
    train['Transaction_hour'] = np.floor(train['TransactionDT'] / 3600) % 24

    test['Transaction_day_of_week'] = np.floor((test['TransactionDT'] / (3600 * 24) - 1) % 7)
    test['Transaction_hour'] = np.floor(test['TransactionDT'] / 3600) % 24

    id_cols = ['addr1', 'addr2', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'P_emaildomain']

    gc.collect()

    cols1 = [x for x in train.columns if 'card' in x and '_fe1' in x]

    cols1 = [x for x in train.columns if 'card' in x and '_fe1' in x]
    ft = 'N1'
    train[ft] = train.groupby(cols1)['TransactionDT'].transform('count')
    test[ft] = test.groupby(cols1)['TransactionDT'].transform('count')
    # dict_fe, _d_type = freq_encoder2(train, ft, 0)
    # dict_fe_test = freq_encoder2_test(test, ft, dict_fe)
    # train[ft] = train[ft].apply(lambda x: dict_fe[x]).astype(_d_type)
    # test[ft] = test[ft].apply(lambda x: dict_fe_test[x]).astype(_d_type)

    cols1 = ['addr1', 'addr2']
    ft = 'N2'
    train[ft] = train.groupby(cols1)['TransactionDT'].transform('count')
    test[ft] = test.groupby(cols1)['TransactionDT'].transform('count')

    cols1 = [x for x in train.columns if 'C' in x and 'Product' not in x]
    ft = 'N3'
    train[ft] = train.groupby(cols1)['TransactionDT'].transform('count')
    test[ft] = test.groupby(cols1)['TransactionDT'].transform('count')

    cols1 = [x for x in train.columns if 'mail' in x]
    ft = 'N4'
    train[ft] = train.groupby(cols1)['TransactionDT'].transform('count')
    test[ft] = test.groupby(cols1)['TransactionDT'].transform('count')

    cols1 = ['P_emaildomain', 'R_emaildomain', 'addr1']
    ft = 'N5'
    train[ft] = train.groupby(cols1)['TransactionDT'].transform('count')
    test[ft] = test.groupby(cols1)['TransactionDT'].transform('count')

    cols1 = ['P_emaildomain', 'R_emaildomain', 'addr1', 'card1']
    ft = 'N6'
    train[ft] = train.groupby(cols1)['TransactionDT'].transform('count')
    test[ft] = test.groupby(cols1)['TransactionDT'].transform('count')

    cols1 = ['addr1', 'addr2', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6', 'P_emaildomain']
    ft = 'N7'
    train[ft] = train.groupby(cols1)['TransactionDT'].transform('count')
    test[ft] = test.groupby(cols1)['TransactionDT'].transform('count')

    #### V columns

    train['N8'] = train.groupby(cols1)['TransactionDT'].transform('count')
    train['N8'] = (train.N8/train.N8.max()).astype(np.float32).replace(np.inf, np.nan)
    train['N9'] = train.groupby(cols2)['TransactionDT'].transform('count')
    train['N9'] = (train.N9/train.N9.max()).astype(np.float32).replace(np.inf, np.nan)
    train['N10'] = train.groupby(cols3)['TransactionDT'].transform('count')
    train['N10'] = (train.N10/train.N10.max()).astype(np.float32).replace(np.inf, np.nan)
    train['N11'] = train.groupby(cols4)['TransactionDT'].transform('count')
    train['N11'] = (train.N11/train.N11.max()).astype(np.float32).replace(np.inf, np.nan)
    train['N12'] = train.groupby(cols5)['TransactionDT'].transform('count')
    train['N12'] = (train.N12/train.N12.max()).astype(np.float32).replace(np.inf, np.nan)
    train['N13'] = train.groupby(cols6)['TransactionDT'].transform('count')
    train['N13'] = (train.N13/train.N13.max()).astype(np.float32).replace(np.inf, np.nan)
    train['N14'] = train.groupby(cols7)['TransactionDT'].transform('count')
    train['N14'] = (train.N14/train.N14.max()).astype(np.float32).replace(np.inf, np.nan)
    train['N15'] = train.groupby(cols8)['TransactionDT'].transform('count')
    train['N15'] = (train.N15/train.N15.max()).astype(np.float32).replace(np.inf, np.nan)
    train['N16'] = train.groupby(cols9)['TransactionDT'].transform('count')
    train['N16'] = (train.N16/train.N16.max()).astype(np.float32).replace(np.inf, np.nan)
    train['N17'] = train.groupby(cols10)['TransactionDT'].transform('count')
    train['N17'] = (train.N17/train.N17.max()).astype(np.float32).replace(np.inf, np.nan)
    train['N18'] = train.groupby(cols11)['TransactionDT'].transform('count')
    train['N18'] = (train.N18/train.N18.max()).astype(np.float32).replace(np.inf, np.nan)
    train['N19'] = train.groupby(bin_v)['TransactionDT'].transform('count')
    train['N19'] = (train.N19/train.N19.max()).astype(np.float32).replace(np.inf, np.nan)
    n = 20
    for c in V_cols[:4]:
        train['N{}'.format(n)] = (train.groupby(c)['TransactionDT'].transform('std')/train.groupby(c)['TransactionDT'].transform('mean')).replace(np.inf, np.nan)
        n += 1

    test['N8'] = test.groupby(cols1)['TransactionDT'].transform('count')
    test['N8'] = (test.N8/test.N8.max()).astype(np.float32).replace(np.inf, np.nan)
    test['N9'] = test.groupby(cols2)['TransactionDT'].transform('count')
    test['N9'] = (test.N9/test.N9.max()).astype(np.float32).replace(np.inf, np.nan)
    test['N10'] = test.groupby(cols3)['TransactionDT'].transform('count')
    test['N10'] = (test.N10/test.N10.max()).astype(np.float32).replace(np.inf, np.nan)
    test['N11'] = test.groupby(cols4)['TransactionDT'].transform('count')
    test['N11'] = (test.N11/test.N11.max()).astype(np.float32).replace(np.inf, np.nan)
    test['N12'] = test.groupby(cols5)['TransactionDT'].transform('count')
    test['N12'] = (test.N12/test.N12.max()).astype(np.float32).replace(np.inf, np.nan)
    test['N13'] = test.groupby(cols6)['TransactionDT'].transform('count')
    test['N13'] = (test.N13/test.N13.max()).astype(np.float32).replace(np.inf, np.nan)
    test['N14'] = test.groupby(cols7)['TransactionDT'].transform('count')
    test['N14'] = (test.N14/test.N14.max()).astype(np.float32).replace(np.inf, np.nan)
    test['N15'] = test.groupby(cols8)['TransactionDT'].transform('count')
    test['N15'] = (test.N15/test.N15.max()).astype(np.float32).replace(np.inf, np.nan)
    test['N16'] = test.groupby(cols9)['TransactionDT'].transform('count')
    test['N16'] = (test.N16/test.N16.max()).astype(np.float32).replace(np.inf, np.nan)
    test['N17'] = test.groupby(cols10)['TransactionDT'].transform('count')
    test['N17'] = (test.N17/test.N17.max()).astype(np.float32).replace(np.inf, np.nan)
    test['N18'] = test.groupby(cols11)['TransactionDT'].transform('count')
    test['N18'] = (test.N18/test.N18.max()).astype(np.float32).replace(np.inf, np.nan)
    test['N19'] = test.groupby(bin_v)['TransactionDT'].transform('count')
    test['N19'] = (test.N19/test.N19.max()).astype(np.float32).replace(np.inf, np.nan)
    n = 20
    for c in V_cols[:4]:
        test['N{}'.format(n)] = (test.groupby(c)['TransactionDT'].transform('std')/test.groupby(c)['TransactionDT'].transform('mean')).replace(np.inf, np.nan)
        n += 1


    c_cols = [x for x in train.columns if 'C' in x and x not in ['ProductCD','ProductCD_fe1','ProductCD_fe2']]

    train['R1'] = (np.log1p(train.C13/train.C1)).replace(np.inf, np.nan)

    train['R2'] = (np.log1p(train.D1/train.D15)).replace(np.inf, np.nan)

    train['R3'] = (np.log1p(train.TransactionAmt/train.TransactionAmt.values.mean())).replace(np.inf, np.nan)

    train['R4'] = (np.log1p(train.N3/train.N1)).replace(np.inf, np.nan)

    train['R5'] = (train['D15'] / train.groupby(['card1', 'addr1'])['D15'].transform('std')).replace(np.inf, np.nan)

    train['R6'] = (train['D15'] / train.groupby(['TransactionDT'])['D15'].transform('std')).replace(np.inf, np.nan)

    train['R7'] = (train.TransactionAmt / train.Transaction_hour / train.Transaction_day_of_week).replace(np.inf, np.nan)

    train['R8'] = (train['TransactionAmt'] / train.groupby(['card1', 'addr1'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

    train['R9'] = (train['TransactionAmt'] / train.groupby(['card1','C13','C1','card2','addr1'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

    train['R10'] = (train['TransactionAmt'] / train.groupby(['card1','addr1','id_01','V258','D1', 'P_emaildomain', 'R_emaildomain'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

    train['R11'] = (train['TransactionAmt'] / train.groupby(['card1','addr1','P_emaildomain', 'R_emaildomain'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

    train['R12'] = (train.groupby(['card1','addr1'])['card1'].transform('mean') * train.groupby(['card1','addr1'])['card1'].transform('count')).replace(np.inf, np.nan)

    train['R13'] = (train.TransactionAmt / train.groupby(['card1', 'card2','addr1'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

    train['R14'] = (train.C2 / train.groupby(['card1', 'V258', 'D10'])['C2'].transform('mean')).replace(np.inf, np.nan)

    train['R15'] = (train.V130 / train.groupby(['addr1', 'card1', 'C10', 'Transaction_hour', 'Transaction_day_of_week'])['V130'].transform('std')).replace(np.inf, np.nan)

    train['R16'] = (train.V258 / train.groupby(['addr1', 'card1', 'C10', 'Transaction_hour', 'Transaction_day_of_week'])['V258'].transform('std')).replace(np.inf, np.nan)

    train['R17'] = (train.D10 / train.groupby(id_cols)['D10'].transform('std')).replace(np.inf, np.nan)

    train['R18'] = (train.D11 / train.groupby(id_cols)['D11'].transform('std')).replace(np.inf, np.nan)

    train['R19'] = (train.D15 / train.groupby(id_cols)['D15'].transform('std')).replace(np.inf, np.nan)

    train['R20'] = (train.D3 / train.groupby(id_cols)['D3'].transform('std')).replace(np.inf, np.nan)

    train['R21'] = (train.D4 / train.groupby(id_cols)['D4'].transform('std')).replace(np.inf, np.nan)

    train['R22'] = (train.D5 / train.groupby(id_cols)['D5'].transform('std')).replace(np.inf, np.nan)

    train['R23'] = (train.D6 / train.groupby(id_cols)['D6'].transform('std')).replace(np.inf, np.nan)

    train['R24'] = train.D7 * train.groupby(id_cols)['D7'].transform('std')
    train['R24'] = train['R24'].replace(np.inf, np.nan)
    train['R24'] = train['R24']/train['R24'].max()

    train['R25'] = train.D12 * train.groupby(id_cols)['D12'].transform('std')
    train['R25'] = train['R25'].replace(np.inf, np.nan)
    train['R25'] = train['R25']/train['R25'].max()

    train['R26'] = train.D6 * train.groupby(id_cols)['D6'].transform('std')
    train['R26'] = train['R26'].replace(np.inf, np.nan)
    train['R26'] = train['R26']/train['R26'].max()

    train['R27'] = train.D8 * train.groupby(id_cols)['D8'].transform('std')
    train['R27'] = train['R27'].replace(np.inf, np.nan)
    train['R27'] = train['R27']/train['R27'].max()

    train['R28'] = (train.TransactionAmt - train.groupby(id_cols)['TransactionAmt'].shift(1)).replace(np.inf, np.nan)

    train['R29'] = (train.TransactionDT - (train.TransactionDT - train.groupby(id_cols)['TransactionDT'].shift(1))).replace(np.inf, np.nan)

    train['R30'] = (train.dist1 - train.groupby(id_cols)['dist1'].shift(1)).replace(np.inf, np.nan)

    train['R31'] = (train.dist2 - train.groupby(id_cols)['dist2'].shift(1)).replace(np.inf, np.nan)

    n = 32
    for c in c_cols:
        train['R{}'.format(n)] = (train[c] / train.groupby(id_cols)[c].transform('std')).replace(np.inf, np.nan)
        n += 1

    for c in c_cols:
        train['R{}'.format(n)] = (train[c] / train.groupby(id_cols)[c].transform('mean')).replace(np.inf, np.nan)
        n += 1

    for c in c_cols:
        train['R{}'.format(n)] = (train[c] / (train.groupby(id_cols)[c].transform('std')/train.groupby(id_cols)[c].transform('mean'))).replace(np.inf, np.nan)
        n += 1


    test['R1'] = (np.log1p(test.C13/test.C1)).replace(np.inf, np.nan)

    test['R2'] = (np.log1p(test.D1/test.D15)).replace(np.inf, np.nan)

    test['R3'] = (np.log1p(test.TransactionAmt/test.TransactionAmt.values.mean())).replace(np.inf, np.nan)

    test['R4'] = (np.log1p(test.N3/test.N1)).replace(np.inf, np.nan)

    test['R5'] = (test['D15'] / test.groupby(['card1', 'addr1'])['D15'].transform('std')).replace(np.inf, np.nan)

    test['R6'] = (test['D15'] / test.groupby(['TransactionDT'])['D15'].transform('std')).replace(np.inf, np.nan)

    test['R7'] = (test.TransactionAmt / test.Transaction_hour / test.Transaction_day_of_week).replace(np.inf, np.nan)

    test['R8'] = (test['TransactionAmt'] / test.groupby(['card1', 'addr1'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

    test['R9'] = (test['TransactionAmt'] / test.groupby(['card1','C13','C1','card2','addr1'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

    test['R10'] = (test['TransactionAmt'] / test.groupby(['card1','addr1','id_01','V258','D1', 'P_emaildomain', 'R_emaildomain'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

    test['R11'] = (test['TransactionAmt'] / test.groupby(['card1','addr1','P_emaildomain', 'R_emaildomain'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

    test['R12'] = (test.groupby(['card1','addr1'])['card1'].transform('mean') * test.groupby(['card1','addr1'])['card1'].transform('count')).replace(np.inf, np.nan)

    test['R13'] = (test.TransactionAmt / test.groupby(['card1', 'card2','addr1'])['TransactionAmt'].transform('std')).replace(np.inf, np.nan)

    test['R14'] = (test.C2 / test.groupby(['card1', 'V258', 'D10'])['C2'].transform('mean')).replace(np.inf, np.nan)

    test['R15'] = (test.V130 / test.groupby(['addr1', 'card1', 'C10', 'Transaction_hour', 'Transaction_day_of_week'])['V130'].transform('std')).replace(np.inf, np.nan)

    test['R16'] = (test.V258 / test.groupby(['addr1', 'card1', 'C10', 'Transaction_hour', 'Transaction_day_of_week'])['V258'].transform('std')).replace(np.inf, np.nan)

    test['R17'] = (test.D10 / test.groupby(id_cols)['D10'].transform('std')).replace(np.inf, np.nan)

    test['R18'] = (test.D11 / test.groupby(id_cols)['D11'].transform('std')).replace(np.inf, np.nan)

    test['R19'] = (test.D15 / test.groupby(id_cols)['D15'].transform('std')).replace(np.inf, np.nan)

    test['R20'] = (test.D3 / test.groupby(id_cols)['D3'].transform('std')).replace(np.inf, np.nan)

    test['R21'] = (test.D4 / test.groupby(id_cols)['D4'].transform('std')).replace(np.inf, np.nan)

    test['R22'] = (test.D5 / test.groupby(id_cols)['D5'].transform('std')).replace(np.inf, np.nan)

    test['R23'] = (test.D6 / test.groupby(id_cols)['D6'].transform('std')).replace(np.inf, np.nan)

    test['R24'] = test.D7 * test.groupby(id_cols)['D7'].transform('std')
    test['R24'] = test['R24'].replace(np.inf, np.nan)
    test['R24'] = test['R24']/test['R24'].max()

    test['R25'] = test.D12 * test.groupby(id_cols)['D12'].transform('std')
    test['R25'] = test['R25'].replace(np.inf, np.nan)
    test['R25'] = test['R25']/test['R25'].max()

    test['R26'] = test.D6 * test.groupby(id_cols)['D6'].transform('std')
    test['R26'] = test['R26'].replace(np.inf, np.nan)
    test['R26'] = test['R26']/test['R26'].max()

    test['R27'] = test.D8 * test.groupby(id_cols)['D8'].transform('std')
    test['R27'] = test['R27'].replace(np.inf, np.nan)
    test['R27'] = test['R27']/test['R27'].max()

    test['R28'] = (test.TransactionAmt - test.groupby(id_cols)['TransactionAmt'].shift(1)).replace(np.inf, np.nan)

    test['R29'] = (test.TransactionDT - (test.TransactionDT - test.groupby(id_cols)['TransactionDT'].shift(1))).replace(np.inf, np.nan)

    test['R30'] = (test.dist1 - test.groupby(id_cols)['dist1'].shift(1)).replace(np.inf, np.nan)

    test['R31'] = (test.dist2 - test.groupby(id_cols)['dist2'].shift(1)).replace(np.inf, np.nan)

    n = 32
    for c in c_cols:
        test['R{}'.format(n)] = (test[c] / test.groupby(id_cols)[c].transform('std')).replace(np.inf, np.nan)
        n += 1

    for c in c_cols:
        test['R{}'.format(n)] = (test[c] / test.groupby(id_cols)[c].transform('mean')).replace(np.inf, np.nan)
        n += 1

    for c in c_cols:
        test['R{}'.format(n)] = (test[c] / (test.groupby(id_cols)[c].transform('std')/test.groupby(id_cols)[c].transform('mean'))).replace(np.inf, np.nan)
        n += 1

    
    gc.collect()
    
    print('End!\n')
    
    return train, test

In [36]:
# df = process_data(df)

In [38]:
new_cols = list(set(df.columns) - set(old_cols))
len(new_cols)

149

In [41]:
new_cols = list(set(df.columns) - set(old_cols))
len(new_cols)

146

In [48]:
dict_types = reduce_memory2(train[new_cols])

Reduce_memory...


In [39]:
dict_types = reduce_memory2(df[new_cols])

Reduce_memory...


In [46]:
# for i, j in zip(dict_types.keys(), dict_types.values()):
#     train[i] = train[i].astype(j)

In [40]:
for i, j in zip(dict_types.keys(), dict_types.values()):
    df[i] = df[i].astype(j)

In [42]:
dict(df.dtypes)

{'C1': dtype('float16'),
 'C10': dtype('float16'),
 'C11': dtype('float16'),
 'C12': dtype('float16'),
 'C13': dtype('float16'),
 'C14': dtype('float16'),
 'C2': dtype('float16'),
 'C3': dtype('float16'),
 'C4': dtype('float16'),
 'C5': dtype('float16'),
 'C6': dtype('float16'),
 'C7': dtype('float16'),
 'C8': dtype('float16'),
 'C9': dtype('float16'),
 'D1': dtype('float16'),
 'D10': dtype('float16'),
 'D11': dtype('float16'),
 'D12': dtype('float16'),
 'D13': dtype('float16'),
 'D14': dtype('float16'),
 'D15': dtype('float16'),
 'D2': dtype('float16'),
 'D3': dtype('float16'),
 'D4': dtype('float16'),
 'D5': dtype('float16'),
 'D6': dtype('float16'),
 'D7': dtype('float16'),
 'D8': dtype('float16'),
 'DeviceInfo': dtype('uint16'),
 'DeviceType_0': dtype('int8'),
 'DeviceType_1': dtype('int8'),
 'DeviceType_2': dtype('int8'),
 'M1': dtype('uint8'),
 'M2': dtype('uint8'),
 'M3': dtype('uint8'),
 'M4': dtype('uint8'),
 'M5': dtype('uint8'),
 'M6': dtype('uint8'),
 'M7': dtype('uint8'),


In [ ]:
# drop_cols = ['D10', 'D11', 'D12', 'D15', 'D3', 'D4',
#             'D5', 'D6', 'D7', 'D8']

In [ ]:
# df = df[[x for x in df.columns if x not in drop_cols]]

In [ ]:
# df = df.replace(np.inf, np.nan)

In [43]:
train = df.iloc[:train_index, :]
test = df.iloc[train_index:, :]
train.shape, test.shape

((590540, 724), (506691, 724))

In [44]:
train.to_csv(data_folder+'/train_ft_eng_3.csv', header=True, index=None)
print('train guardado')
test.to_csv(data_folder+'/test_ft_eng_3.csv', header=True, index=None)
print('test guardado')

train guardado
test guardado


In [53]:
train.shape, test.shape

((270901, 716), (221908, 716))

In [54]:
train.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,R64,R65,R66,R67,R68,R69,R70,R71,R72,R73
0,0,86400,68.5,0,1749,12,0,3,35,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,86401,29.0,0,139,31,0,1,3,1,...,0.0,NaN,NaN,0.0,0.000,0.0,1.385742,NaN,0.726074,1.069336
2,0,86469,59.0,0,101,3,0,0,2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,86499,50.0,0,17,20,0,1,4,0,...,NaN,13.851562,NaN,0.0,5.125,0.0,4.273438,NaN,56.156250,10.195312
4,0,86506,50.0,3,5782,7,0,1,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
